<a href="https://colab.research.google.com/github/samadpls/TensorFlow-Model-Exploration/blob/colab/Natural_Language_Processing_With_RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [6]:
with open(path_to_file,'rb') as file:
  text = file.read().decode(encoding='utf-8')
print(len(text))
print(text[:1000])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for re

In [7]:
vocab =  sorted(set(text))
char2idx =  {u:i for i,u in enumerate(vocab)}
idx2char =  np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int =  text_to_int(text)

In [10]:
print("Text-->",text[:14])
print("Encoded--> ", text_to_int(text[:14]))

Text--> First Citizen:
Encoded-->  [18 47 56 57 58  1 15 47 58 47 64 43 52 10]


In [11]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return "".join(idx2char[ints])

print(int_to_text(text_to_int(text[:14])))

First Citizen:


In [12]:
seq_length = 100
examples_per_epoch =  len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [13]:
sequences =  char_dataset.batch(seq_length+1, drop_remainder=True)

In [14]:
def split_input_target(chunk):
  input_text = chunk[:1]
  target_text =  chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [15]:
for x,y in dataset.take(2):
  print("Example")
  print("input")
  print(int_to_text(x))
  print('output')
  print(int_to_text(y))

Example
input
F
output
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 
Example
input
a
output
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [16]:
batch_size = 64
vocab_size = len(vocab)
embeding_dim = 256
rnn_units = 1024

buffer_size = 10000

data = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder= True)

In [21]:
def build_model(vocab_size, embeding_dim, rnn_units, batch_size):
  model =  tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size,embeding_dim,
      batch_input_shape= [batch_size,None]),
      tf.keras.layers.LSTM(rnn_units,
                           return_sequences=True, # want on every intermediate part
                           stateful=True,
                           recurrent_initializer='glorot_uniform'),
      tf.keras.layers.Dense(vocab_size)
      ])
  return model
model =  build_model(vocab_size,embeding_dim,rnn_units,batch_size)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5330241 (20.33 MB)
Trainable params: 5330241 (20.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
for input_example, target_example in data.take(1):
  example_prd= model(input_example)
  print(example_prd.shape,"batch size, sequence length, vocab_size")

(64, 1, 65) batch size, sequence length, vocab_size


In [23]:
print(len(example_prd))
print(example_prd)

64
tf.Tensor(
[[[ 5.0332240e-04  2.8083497e-03 -1.2157870e-03 ...  2.0423471e-03
    9.4294082e-05  1.7096852e-03]]

 [[ 1.2716302e-03 -5.1937285e-03  2.2551650e-03 ...  1.7016797e-03
    5.7998821e-03 -8.0865882e-03]]

 [[-1.6213255e-03 -3.9232830e-03  2.0987634e-03 ... -3.2209423e-03
   -4.5928489e-03  8.2668883e-04]]

 ...

 [[ 3.8223004e-03 -2.9790751e-04 -3.7611085e-03 ... -1.2006077e-03
    8.3139213e-04 -1.8752748e-03]]

 [[-1.2047737e-03  2.0690700e-03 -3.3193519e-03 ...  2.8579407e-03
   -4.0857475e-03 -8.7283412e-03]]

 [[ 5.0332240e-04  2.8083497e-03 -1.2157870e-03 ...  2.0423471e-03
    9.4294082e-05  1.7096852e-03]]], shape=(64, 1, 65), dtype=float32)


In [24]:
def loss(labels,logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels,logits,from_logits=True)

In [25]:
model.compile(optimizer='adam', loss=loss)

In [26]:
checkpoint =  './training_checkpoints'
chckpoint_prefix = os.path.join(checkpoint,'ckpt_{epoch}')

chckpoint_callback= tf.keras.callbacks.ModelCheckpoint(
    filepath=chckpoint_prefix,
    save_weights_only=True
    )

In [32]:
history=  model.fit(data,epochs=0,callbacks=[chckpoint_callback])